# Queries for data processing for CI/ASU Decision Theater presentation

Import GEE api and json library. Note that `ee.initialize` will fail if the key for GEE has not yet been setup

In [1]:
import ee
import json
import io

service_account = 'gef-ldmp-server@gef-ld-toolbox.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'dt_key.json')
ee.Initialize(credentials)
#ee.Initialize()

# Define a geojson as a string to use for testing
geojson_text = '''{"type": "Polygon", "coordinates": [ [ [ 36.898123159960932, -0.220252698724199 ], [ 37.40818204121706, 0.915787536800825 ], [ 38.613775760549743, 0.011592247301316 ], [ 38.219639352306366, -0.892603042198193 ], [ 36.898123159960932, -0.220252698724199 ] ] ] }'''
aoi = ee.Geometry(json.loads(geojson_text))

out = {}

## Get statistics on areas degraded within a polygon

In [2]:
# Function to pull areas that are saved as properties within a feature class,
# convert them to percentages of the total area, and return as a dictionary. Sums
# all features together. Scaling converts to percentages if set to 100 and
# scaling is True
def fc_areas_to_pct_dict(fc, normalize=True, scaling=100):
    # Note that there may be multiple features
    ret = {}
    for p in [feature['properties'] for feature in fc.getInfo()['features']]:
        areas = {}
        # If there is more than one feature, need to update ret with these values
        for key, value in p.iteritems():
            if key in ret:
                ret[key] += value
            else:
                ret[key] = value
    if normalize:
        denominator = sum(ret.values())
        ret = {key: (value/denominator)*scaling for key, value in ret.iteritems()}
    return ret

In [3]:
# polygon area in hectares
out['area'] = aoi.area().divide(10000).getInfo()

# s2_02: Number of people living inside the polygon in 2015
pop_cnt = ee.Image("CIESIN/GPWv4/unwpp-adjusted-population-count/2015");
population = pop_cnt.reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=1000, maxPixels=1e9)
out['population'] = population.getInfo()['population-count']

# s2_03: Main livelihoods
liv = ee.FeatureCollection("users/geflanddegradation/toolbox_datasets/livelihoodzones")

livImage = liv.filter(ee.Filter.neq('lztype_num', None)).reduceToImage(properties=['lztype_num'], reducer=ee.Reducer.first())

fields = ["Agro-Forestry", "Agro-Pastoral", "Arid", "Crops - Floodzone", "Crops - Irrigated", "Crops - Rainfed", "Fishery", "Forest-Based", "National Park", "Other", "Pastoral", "Urban"]
# multiply pixel area by the area which experienced each of the five transitions --> output: area in ha
livelihoodareas = livImage.eq([1,2,3,4,5,6,7,8,9,10,11,12]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum(),30)
out['livelihoods'] = fc_areas_to_pct_dict(livelihoodareas)

In [4]:
# s3_01: SDG 15.3.1 degradation classes 

te_sdgi = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_sdg1531_gpg_globe_2001_2015_modis")
sdg_areas = te_sdgi.eq([-32768,-1,0,1]).rename(["nodata", "degraded", "stable", "improving"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['area_sdg'] = fc_areas_to_pct_dict(sdg_areas)

# s3_02: Productivity degradation classes
te_prod = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lp7cl_globe_2001_2015_modis").remap([-32768,1,2,3,4,5,6,7],[-32768,-1,-1,0,0,0,1,1])
prod_areas = te_prod.eq([-32768,-1,0,1]).rename(["nodata", "decline", "stable", "improvement"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['area_prod'] = fc_areas_to_pct_dict(prod_areas)

# s3_03: Land cover degradation classes
te_land = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lc_traj_globe_2001-2001_to_2015")
lc_areas = te_land.select("lc_dg").eq([-1,0,1]).rename(["degradation", "stable", "improvement"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['area_lc'] = fc_areas_to_pct_dict(lc_areas)

# s3_04: soc degradation classes
te_socc_deg = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_soc_globe_2001-2015_deg").select("soc_deg")
soc_areas = te_socc_deg.eq([-32768,-1,0,1]).rename(["no data", "degradation", "stable", "improvement"]).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['area_soc'] = fc_areas_to_pct_dict(soc_areas)

In [5]:
# s3_05: compute land cover classes for 2001 and 2015, and the transitions which occured
te_land = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lc_traj_globe_2001-2001_to_2015").select("lc_tr")
te_prod = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lp7cl_globe_2001_2015_modis").remap([-32768,1,2,3,4,5,6,7],[-32768,-1,-1,0,0,0,1,1])

fields = ["nodata", "decline", "stable", "improvement"]

prod_forests = te_prod.updateMask(te_land.eq(11)).eq([-32768,-1,0,1]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['prod_forests'] = fc_areas_to_pct_dict(prod_forests)

prod_grasslands = te_prod.updateMask(te_land.eq(22)).eq([-32768,-1,0,1]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['prod_grasslands'] = fc_areas_to_pct_dict(prod_grasslands)

prod_agriculture = te_prod.updateMask(te_land.eq(33)).eq([-32768,-1,0,1]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['prod_agriculture'] = fc_areas_to_pct_dict(prod_forests)


In [6]:
# s3_06: compute land cover classes for 2001 and 2015, and the transitions which occured
te_land = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lc_traj_globe_2001-2001_to_2015")

# field names for annual land covers
fields = ["forest", "grassland", "agriculture", "wetlands", "artificial", "other land-bare", "water"]

# multiply pixel area by the area which experienced each of the lc classes --> output: area in ha
lc_baseline = te_land.select("lc_bl").eq([1,2,3,4,5,6,7]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
# multiply pixel area by the area which experienced each of the lc classes --> output: area in ha
lc_target = te_land.select("lc_tg").eq([1,2,3,4,5,6,7]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())

out['lc_2001'] = fc_areas_to_pct_dict(lc_baseline)
out['lc_2015'] = fc_areas_to_pct_dict(lc_target)

# field names for land cover transitions between 2001-2015
fields = ["for-for", "for-gra", "for-agr", "for-wet", "for-art", "for-oth", "for-wat",
          "gra-for", "gra-gra", "gra-agr", "gra-wet", "gra-art", "gra-oth", "gra-wat",
          "agr-for", "agr-gra", "agr-agr", "agr-wet", "agr-art", "agr-oth", "agr-wat",
          "wet-for", "wet-gra", "wet-agr", "wet-wet", "wet-art", "wet-oth", "wet-wat",
          "art-for", "art-gra", "art-agr", "art-wet", "art-art", "art-oth", "art-wat",
          "oth-for", "oth-gra", "oth-agr", "oth-wet", "oth-art", "oth-oth", "oth-wat",
          "wat-for", "wat-gra", "wat-agr", "wat-wet", "wat-art", "wat-oth", "wat-wat"]

# multiply pixel area by the area which experienced each of the lc transition classes --> output: area in ha
lc_transitions = te_land.select("lc_tr").eq([11,12,13,14,15,16,17,21,22,23,24,25,26,26,31,32,33,34,35,36,37,41,42,43,44,45,46,47,51,52,53,54,55,56,57,
              61,62,63,64,65,66,67,71,72,73,74,75,76,77]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
out['lc_transition'] = fc_areas_to_pct_dict(lc_transitions)


In [7]:
# s3_07: percent change in soc stocks between 2001-2015
soc_pch_img = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_soc_globe_2001-2015_deg").select("soc_pch")

# compute statistics for region
soc_pch = soc_pch_img.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi, scale=250, maxPixels=1e9)
# Multiple by 100 to convert to a percentage
out['soc_change_percent'] = soc_pch.getInfo()['soc_pch'] * 100
  
# s3_08: change in soc stocks in tons of co2 eq between 2001-2015
soc_an_img = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_soc_globe_2001-2015_annual_soc")

# compute change in SOC between 2001 and 2015 converted to co2 eq
soc_chg_an = (soc_an_img.select('y2015').subtract(soc_an_img.select('y2001'))).multiply(250*250/10000).multiply(3.67)
# compute statistics for the region
soc_chg_tons_co2e = soc_chg_an.reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=250, maxPixels=1e9)
out['soc_change_tons_co2e'] = soc_chg_tons_co2e.getInfo()['y2015']

In [8]:
with io.open('out.json', 'w', encoding='utf-8') as f:

print out
  f.write(json.dumps(d, ensure_ascii=False, indent=4))

IndentationError: expected an indented block (<ipython-input-8-7847e7045098>, line 3)